<div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 40px; margin-top: 0;">
    <div style="flex: 0 0 auto; margin-left: 0; margin-bottom: 0;margin-top: 0;">
        <img src="./pics/NationalDataPlatform_logo.png" alt="NationalDataPlatform Logo" style="width: 179px; margin-bottom: 0px;">
    </div>    
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 0;">
        <img src="./pics/logo_UCSD.png" alt="UCSD Logo" style="width: 179px; margin-bottom: 0px; margin-top: 20px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/sdsclogo-plusname-horiz-red.jpg" alt="San Diego Supercomputer Center Logo" width="300"/>
    </div>
</div>
<h1 style="text-align: center; font-size: 24px; margin-top: 0;">NSF National Data Platform (NDP)</h1>
<h3 style="text-align: center; font-size: 18px; margin-top: 10px;">LLM as a Service Tutorial</h3>
<div style="margin: 20px 0;">
    <p align="justify"> Large Language Models are a powerful AI tool with multiple applications in both research and education, given their capacity to process big amounts of information, providing human-like answers which are tuned for different audiences.</p>
    <p align="justify"> Understanding today's relevance of LLM's, the National Data Platform (NDP) has developed an LLM service to contribute to the research and education goals of its users.</p>
    <p align="justify"> In this guide, we are covering the use of an LLM as an NDP service. The main purpose of this demo is to showcase how this service works, and allow the user to identify the potential in the use of this service in research and education.</p>
</div>

<center>
    <div style="text-align: right; padding: 5px;">
        <p style="text-align: right;"><strong>Contact:</strong><a href="https://docs.google.com/forms/d/e/1FAIpQLSfzjlc0Sw2fTFTKArOZ0ffKNdVcPivf218kLXkBKfobGPbDMw/viewform"> NDP Issue Reporting Form </a></p>
    </div>
</cente


<div style="display: flex; align-items: center; justify-content: flex-start; margin-top: 20px; border-top: 1px solid #ccc; padding-top: 20px;">
    <img src="https://new.nsf.gov/themes/custom/nsf_theme/components/images/logo/logo-desktop.svg" alt="NSF Logo" style="width: 120px; margin-right: 10px;">
    <p style="font-size: 12px;">The National Data Platform was funded by NSF 2333609 under CI, CISE Research Resources programs. Any opinions, findings, conclusions, or recommendations expressed in this material are those of the author(s) and do not necessarily reflect the views of the funders.</p>
</div>
<hr>

### What is an LLM?

An LLM, or Large Language Model, is a type of artificial intelligence designed to understand and generate human-like text based on the data it's been trained on. By adding a vast amount of text from different sources and context's (web, books, papers, among others) LLM's are capable to identify the patterns of the human language under different contexts and provide responses to complex questions.  Popular LLM's today include GPT-4 (ChatGPT), LlaMA-2, Claude-2, among others. 

LLM's posses a huge potential in both research and education, given their capability to quickly process and summarize a vast amount of information. LLM's can bee seen as a powerful tool to accelerate learning, facilitate the sharing of knowledge, process and generate big amounts of data, generate new hypothesis questions, among other uses.

### ClimateGPT

For this demo, we are using the [ClimateGPT](https://climategpt.ai/) model. Climate GPT was launched in January 2024, and it represents one of the first LLM's fine-tuned on climate research chorpus. 

The model is a fine tuned version of META's LlaMA-2, and it has been designed to accelerate research aimed to face the current climate crisis, targeting researcers, policymakers and educators as primary users.

### Hands on

We will start using LLM as a service. First, we will make some questions to the vanilla ClimateGPT to see its capabilities. Then, we will extend the chorpus of the model by adding a new document. We expect the model to be able to answer questions based on the new document.

In [1]:
# Importing the relevant libraries
import requests
import os
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.document_loaders import PyPDFLoader

MODEL = 'eci-io/climategpt-7b' # We specify the model tu use
os.environ['OPENAI_API_BASE'] = 'http://localhost:8000/v1'
os.environ['OPENAI_API_KEY'] = 'EMPTY'

### Test that LLM API is up

In [2]:
!curl  -X 'GET' 'http://localhost:8000/v1/models' -H 'accept: application/json'

{"object":"list","data":[{"id":"ECarbenia/grimoiresigils","object":"model","created":1710722296,"owned_by":"fastchat","root":"ECarbenia/grimoiresigils","parent":null,"permission":[{"id":"modelperm-ovGGEHJJioyjcx7Je9ymSj","object":"model_permission","created":1710722296,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":true,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]},{"id":"eci-io/climategpt-7b","object":"model","created":1710722296,"owned_by":"fastchat","root":"eci-io/climategpt-7b","parent":null,"permission":[{"id":"modelperm-WFVzzLMKjFn67onJvDnG53","object":"model_permission","created":1710722296,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":true,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]},{"id":"text-embedding-ada-002","object":"model","created":1710722296,"owned_by":"fastchat","roo

### The conversation code

In [3]:
def save_answer(question, answer):
    with open('output.txt', 'a') as file:
        file.write(question+'\n')
        file.write(answer+'\n\n')
    

def question(question, chorpus):
    url = "http://localhost:8000/v1/chat/completions"
    
    headers = {
        "Content-Type": "application/json",
    }
    
    data = {
        "model": MODEL,
        "messages": [{"role": "user", "content": question}],
    }
    if chorpus:
        answer = index.query(question, llm=llm)
    else:
        response = requests.post(url, json=data, headers=headers)
        answer = response.json()['choices'][0]['message']['content']
    
    print(f"\nAnswer: {answer}")

    save = input("\n\nWould you like to save your answer? (y/n): ")
    print('\n')
    if save=='y':
        save_answer(question, answer)

def conversation(chorpus=False):
    while True:
        user_input = input("Question ('q' to quit): ")
        if user_input=='q':
            break
        question(user_input, chorpus)

### Run the conversation

Feel free to ask any question you consider relevant

In [4]:
conversation()

Question ('q' to quit):  List the facilities that a NAIRR resource should provide with respect to (a) computing, (b) AI/ML models, and (c) data.



Answer: A NAIRR (National Artificial Intelligence Resource) resource should provide the following facilities with respect to (a) computing, (b) AI/ML models, and (c) data:

(a) Computing:

* High-performance computing resources, such as supercomputers, cloud computing services, or high-performance computing clusters.
* Access to specialized software and tools, such as programming languages, data analysis tools, machine learning libraries, and simulation software.
* Data storage solutions, such as data warehouses, data lakes, or distributed file systems.
* Networking infrastructure, such as high-speed internet connectivity, virtual private networks, or cloud-based networking services.

(b) AI/ML models:

* Access to a diverse range of pre-trained AI/ML models, such as deep learning models, natural language processing models, computer vision models, and reinforcement learning models.
* Regular updates and upgrades to the AI/ML models, to ensure that they are up-to-date with the latest r



Would you like to save your answer? (y/n):  n


Question ('q' to quit):  What task forces are recommended in the report?



Answer: I'm sorry, but without knowing which report you are referring to, I am unable to provide you with an accurate answer. Can you please provide more context or information about the report you are asking about?




Would you like to save your answer? (y/n):  n


Question ('q' to quit):  q


### Example of using ClimateGPT LLM adding documents

Now we will extend the capacity of the model by adding new documentation, which we will feed the model with the relevant context to answer our questions of interest. We will use the following document:

- ["Strengthening and Democratizing the U.S. Artificial Intelligence Innovation Ecosystem: An Implementation Plan for a National Artificial Intelligence Research Resource" (NAIRR-TF-Final-Report-2023.pdf)](chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://www.ai.gov/wp-content/uploads/2023/01/NAIRR-TF-Final-Report-2023.pdf)

The questions that we will use to are the following:

* List the facilities that a NAIRR resource should provide with respect to (a) computing, (b) AI/ML models, and (c) data.  
* What committees does the report refer to?  
* What are NAIRR's goals with respect to human capital?  
* What task forces are recommended in the report?  

In [5]:
# We load the new document to the model
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")
file = '../PDF Documents/NAIRR-TF-Final-Report-2023.pdf'
loader = PyPDFLoader(file)
index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
llm = ChatOpenAI(model=MODEL)

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
# We add True to indicate the model to make use of the new document
conversation(True)

Question ('q' to quit):  List the facilities that a NAIRR resource should provide with respect to (a) computing, (b) AI/ML models, and (c) data.



Answer: With respect to computing, a NAIRR resource should provide access to a variety of computational resources, including conventional servers, computing clusters, high-performance computing (HPC), cloud computing, and edge computing resources. It should also support access to testbeds for AI research and development.

With respect to AI/ML models, a NAIRR resource should provide access to a range of AI and machine learning models, including pre-trained models and models that can be trained on-site. It should also provide tools and software for developing, testing, and deploying AI and machine learning models.

With respect to data, a NAIRR resource should provide access to a wide range of data sources, including open and protected data. It should also provide tools and software for data analysis, visualization, and management. The data should be co-located with computational resources wherever possible, and data providers should facilitate user access to restricted statistical dat



Would you like to save your answer? (y/n):  n


Question ('q' to quit):  What task forces are recommended in the report?



Answer: The report recommends the formation of task forces to address specific issues related to the National AI Research and Development Strategic Plan (NAIRR). These task forces are as follows:

1. Technology Advisory Board: This is a standing task force that will be responsible for providing advice to the NAIRR on the evolving AI tool landscape and making recommendations on additions or deletions to the NAIRR standard virtual organization.
2. Resource Provider Coordination Task Force: This task force will be responsible for coordinating, multi-agency funding opportunities for resource providers.
3. Training and Support Task Force: This task force will be responsible for developing and implementing training and support materials related to the use of NAIRR resources.
4. Implementation and Evaluation Task Force: This task force will be responsible for incorporating lessons learned from the implementation of the NAIRR into future iterations of the plan.

These task forces will work in



Would you like to save your answer? (y/n):  n


Question ('q' to quit):  q


Looking at the responses, we can confirm the model is given proper answers which are constructed taking the added document as context.